In [1]:
import numpy as np
import pandas as pd
import csv
from collections import defaultdict
import random
from scipy import sparse

ModuleNotFoundError: No module named 'tensorrec'

In [21]:
print('Loading ratings')
with open('/Users/deborahgemellaro/Programming/612/project_2/ml-latest-small/ratings.csv', 'r') as ratings_file:
    ratings_file_reader = csv.reader(ratings_file)
    raw_ratings = list(ratings_file_reader)
    raw_ratings_header = raw_ratings.pop(0)

# Iterate through the input to map MovieLens IDs to new internal IDs
# The new internal IDs will be created by the defaultdict on insertion
movielens_to_internal_user_ids = defaultdict(lambda: len(movielens_to_internal_user_ids))
movielens_to_internal_item_ids = defaultdict(lambda: len(movielens_to_internal_item_ids))
for row in raw_ratings:
    row[0] = movielens_to_internal_user_ids[int(row[0])]
    row[1] = movielens_to_internal_item_ids[int(row[1])]
    row[2] = float(row[2])
n_users = len(movielens_to_internal_user_ids)
n_items = len(movielens_to_internal_item_ids)

Loading ratings


In [22]:
# Shuffle the ratings and split them in to train/test sets 80%/20%
random.shuffle(raw_ratings)  # Shuffles the list in-place
cutoff = int(.8 * len(raw_ratings))
train_ratings = raw_ratings[:cutoff]
test_ratings = raw_ratings[cutoff:]

In [27]:
# This method converts a list of (user, item, rating, time) to a sparse matrix
def interactions_list_to_sparse_matrix(interactions):
    users_column, items_column, ratings_column, _ = zip(*interactions)
    return sparse.coo_matrix((ratings_column, (users_column, items_column)),
                             shape=(n_users, n_items))


# Create sparse matrices of interaction data
sparse_train_ratings = interactions_list_to_sparse_matrix(train_ratings)
sparse_test_ratings = interactions_list_to_sparse_matrix(test_ratings)

In [28]:
# Construct indicator features for users and items
user_indicator_features = sparse.identity(n_users)
item_indicator_features = sparse.identity(n_items)

# Build a matrix factorization collaborative filter model
cf_model = tensorrec.TensorRec(n_components=5)

# Fit the collaborative filter model
print("Training collaborative filter")
cf_model.fit(interactions=sparse_train_ratings,
             user_features=user_indicator_features,
item_features=item_indicator_features)

NameError: name 'tensorrec' is not defined

In [11]:
raw_movie_metadata = pd.read_csv("/Users/deborahgemellaro/Programming/612/project_2/ml-latest-small/movies.csv")

In [13]:
movie_genres_by_internal_id = {}
movie_titles_by_internal_id = {}
for row in raw_movie_metadata:
    row[0] = movielens_to_internal_item_ids[int(row[0])]  # Map to IDs
    row[2] = row[2].split('|')  # Split up the genres
    movie_genres_by_internal_id[row[0]] = row[2]
    movie_titles_by_internal_id[row[0]] = row[1]

# Look at an example movie metadata row
print("Raw metadata example:\n{}\n{}".format(raw_movie_metadata_header, 
                                             raw_movie_metadata[0]))

# Build a list of genres where the index is the internal movie ID and
# the value is a list of [Genre, Genre, ...]
movie_genres = [movie_genres_by_internal_id[internal_id]
                for internal_id in range(n_items)]

# Transform the genres into binarized labels using scikit's MultiLabelBinarizer
movie_genre_features = MultiLabelBinarizer().fit_transform(movie_genres)
n_genres = movie_genre_features.shape[1]
print("Binarized genres example for movie {}:\n{}".format(movie_titles_by_internal_id[0], 
                                                          movie_genre_features[0]))

# Coerce the movie genre features to a sparse matrix, which TensorRec expects
movie_genre_features = sparse.coo_matrix(movie_genre_features)

NameError: name 'movielens_to_internal_item_ids' is not defined